In [2]:
import pandas as pd
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.modules.distance import PairwiseDistance
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data
from tqdm import tnrange
from tqdm import tqdm_notebook

In [14]:
cuda = torch.device('cuda')
cpu = torch.device('cpu')

# The Mighty KNN

In [3]:
def euk2(x,y):
    d = x-y
    return d @ d

In [4]:
def lnorm(p): 
    return lambda x,y: (x-y).pow(p).sum().pow(1/p)

In [5]:
def dist_mat(X, Y, d=None):
    if d == euk2:
        D = X @ Y.T
        return torch.sum(X**2,1,keepdim=True) + torch.sum(Y**2,1,keepdim=True).T - 2*D
    else:
        return torch.Tensor([d(X[i], Y[i]) for i in range(X.shape[0]) for j in range(Y.shape[0])])               

In [66]:
class KNN(nn.Module):
    def __init__(self, X_train, y_train, k=5, morphism = lambda x: x, distance=euk2, batch_size=None):
        
        self.X = morphism(X_train)
        self.y = y_train
        self.M = morphism
        self.d = distance
        self.b = batch_size if batch_size else X_train.shape[0]
        self.k = k
        
    def forward(self, Y, fast=False, verbose=False):
        ind = torch.randperm(self.X.shape[0])[:self.b]
        Y = self.M(Y)
        if fast:
            dm = dist_mat(Y, self.X[ind], self.d)
            _, I = dm.sort(1)
            ret, _ = self.y[ind][I[:,:self.k]].mode(1)
        else:
            ret = torch.zeros(Y.shape[0])
            for i in range(Y.shape[0]):
                if verbose: print("Iteration: ", i)
                a = torch.zeros(self.k, dtype=torch.long)
                d = torch.ones(self.k) * float('inf')
                for j in ind:
                    nd = self.d(Y[i], self.X[j])
                    if d.max() > nd:
                        a[d.argmax()] = j
                        d[d.argmax()] = nd
                ret[i], _ = self.y[a].mode()
        return ret

In [67]:
X = torch.rand(100,10)

In [68]:
y = torch.ceil(torch.rand(100)*5)

In [69]:
Y = torch.rand(20,10)

In [70]:
knn = KNN(X, y)

In [71]:
knn.forward(Y)

tensor([2., 4., 3., 2., 4., 5., 2., 5., 3., 3., 2., 3., 2., 4., 4., 4., 1., 2.,
        2., 3.])

In [72]:
knn.forward(Y, fast=True)

tensor([2., 4., 3., 2., 4., 5., 2., 5., 3., 3., 2., 3., 2., 4., 4., 4., 1., 2.,
        2., 3.])

# TEST

In [73]:
class Images(torch.utils.data.Dataset):
    def __init__(self):
        D = np.r_[pd.read_csv("./mnist1.csv").to_numpy(), pd.read_csv("./mnist2.csv").to_numpy()]
        self.X = torch.Tensor(D[:, :-1].reshape(70000, 1, 28, 28))/255
        self.y = torch.Tensor(D[:, -1:])
        
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx, :, :, :], self.y[idx, 0]
    
    def to(self, device):
        self.X = self.X.to(device)
        self.y = self.y.to(device)

In [74]:
Img = Images()
Img.to(cuda)

In [37]:
Img[:][0].shape

torch.Size([70000, 1, 28, 28])

In [38]:
Img[:69000][0].view(69000, 28*28)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [157]:
ind = torch.randperm(70000)

In [158]:
knn = KNN(Img[ind[:69900]][0].view(69900, 28*28), Img[ind[:69900]][1])

In [159]:
y_pred = knn.forward(Img[ind[69900:]][0].view(100, 28*28), verbose=True, fast=True)

In [160]:
y_pred

tensor([2., 5., 4., 9., 4., 7., 2., 9., 9., 2., 3., 0., 0., 3., 7., 3., 1., 1.,
        0., 1., 7., 3., 2., 7., 4., 0., 9., 4., 8., 2., 5., 2., 7., 5., 7., 2.,
        1., 1., 9., 7., 0., 4., 0., 4., 4., 1., 0., 4., 3., 0., 2., 6., 7., 9.,
        0., 0., 3., 0., 5., 2., 8., 4., 6., 1., 2., 0., 4., 2., 1., 3., 1., 6.,
        8., 3., 6., 8., 2., 0., 0., 1., 5., 0., 9., 1., 3., 0., 9., 3., 7., 7.,
        2., 1., 9., 2., 2., 6., 7., 0., 1., 0.], device='cuda:0')

In [161]:
Img[ind[69900:]][1]

tensor([2., 5., 4., 9., 4., 7., 2., 9., 9., 4., 3., 0., 0., 3., 7., 3., 1., 1.,
        0., 1., 7., 3., 2., 7., 2., 0., 9., 4., 8., 2., 5., 2., 7., 5., 7., 2.,
        1., 1., 9., 7., 0., 4., 0., 4., 4., 1., 0., 4., 3., 0., 2., 3., 7., 9.,
        0., 0., 3., 4., 5., 6., 8., 4., 6., 1., 6., 0., 4., 2., 1., 3., 1., 6.,
        8., 3., 6., 8., 2., 3., 0., 1., 5., 0., 9., 1., 6., 0., 9., 3., 7., 7.,
        8., 1., 9., 2., 2., 4., 7., 0., 1., 0.], device='cuda:0')

In [162]:
(Img[ind[69900:]][1] - y_pred.to(cuda)).eq(0).sum()

tensor(90, device='cuda:0')

In [165]:
def sample():
    ind = torch.randperm(70000)
    knn = KNN(Img[ind[:69900]][0].view(69900, 28*28), Img[ind[:69900]][1])
    y_pred = knn.forward(Img[ind[69900:]][0].view(100, 28*28), verbose=True, fast=True)
    print("hello")
    return (Img[ind[69900:]][1] - y_pred.to(cuda)).eq(0).sum()

In [166]:
torch.Tensor([sample() for i in range(1000)]).mean()

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hell

tensor(86.0610)